In [ ]:
import os

os.environ["PYTHON_GIL"] = "0"

In [ ]:
import asyncio
import datetime
import logging

import dotenv

import london.roads
import tfl.api
import tfl.exceptions
import tfl.models

dotenv.load_dotenv()

In [ ]:
handler = logging.StreamHandler()


class LogFilter(logging.Filter):
    def filter(self, record):
        return "tfl.api" in record.module


# logging.getLogger().addFilter()
handler.addFilter(LogFilter())
logging.getLogger().addHandler(handler)

In [ ]:
tf_client = tfl.api.Tfl(app_key=os.environ["FLATHUNT__TFL_API_KEY"])

stations_facilities = await tf_client.get_stations_facilities()

In [ ]:
print(stations_facilities.stations.station[0].model_dump_json(indent=2))

In [ ]:
# TODO: using this gives me the coordinates and the station IDs so that I can get the timetables!
other_result = await tf_client.get_stop_points_by_mode(tfl.models.ModeId.TUBE)

In [ ]:
print(other_result[0].model_dump_json(indent=2))

In [ ]:
result = await tf_client.get_timetable(
    from_stop_point_id="940GZZLUEUS",
    station_id="victoria",
    direction=tfl.api.Direction.INBOUND,
)

In [ ]:
# TODO: I have the facitilies
# Now I need to get the bus stops - Yue Says NO BUSES

In [ ]:
print(result.model_dump_json(indent=2))

In [ ]:
for route in result.timetable.routes:
    for station_interval in route.station_intervals:
        for interval in station_interval.intervals:
            if interval.stop_id == "940GZZLUGPK":  # Green Park
                print(f"Time to Green Park: {interval.time_to_arrival} mins")

In [ ]:
for route in result.timetable.routes:
    for schedule in route.schedules:
        print(f"Schedule: {schedule.name}")
        for period in schedule.periods or []:
            if period.frequency:
                print(
                    f"  {period.from_time.hour}:{period.from_time.minute} - "
                    f"{period.to_time.hour}:{period.to_time.minute}: "
                    f"every {period.frequency.highest_frequency}-{period.frequency.lowest_frequency} mins"
                )

In [ ]:
import geopandas as gpd

# Load transport point features (stations)
transport_gdf = gpd.read_file(
    "/home/cemlyn/Downloads/greater-london-251126-free.shp/gis_osm_transport_free_1.shp"
)

# Display basic info
print(f"Total transport features: {len(transport_gdf)}")
print(f"\nColumns: {transport_gdf.columns.tolist()}")
print("\nUnique fclass values:")
print(transport_gdf["fclass"].value_counts())


In [ ]:
# Filter for railway stations
railway_stations = transport_gdf[transport_gdf["fclass"] == "railway_station"].copy()

print(f"Total railway stations: {len(railway_stations)}")
print("\nFirst few stations:")
print(railway_stations[["name", "osm_id"]].head(10))

# Get geometry info
railway_stations["lon"] = railway_stations.geometry.x
railway_stations["lat"] = railway_stations.geometry.y

print("\nSample coordinates:")
print(railway_stations[["name", "lon", "lat"]].head())

In [ ]:
railway_stations[["name", "lon", "lat"]].head(60)

In [ ]:
# Also load railway lines to see if we can identify tube lines
railways_gdf = gpd.read_file(
    "/home/cemlyn/Downloads/greater-london-251126-free.shp/gis_osm_railways_free_1.shp"
)

print(f"Total railway features: {len(railways_gdf)}")
print(f"\nColumns: {railways_gdf.columns.tolist()}")
print("\nUnique fclass values:")
print(railways_gdf["fclass"].value_counts())

In [ ]:
# Filter subway lines
subway_lines = railways_gdf[railways_gdf["fclass"] == "subway"].copy()

print(f"Total subway line segments: {len(subway_lines)}")
print("\nUnique subway lines:")
unique_lines = subway_lines["name"].dropna().unique()
print(f"Found {len(unique_lines)} named subway lines")
for line in sorted(unique_lines)[:20]:  # Show first 20
    print(f"  - {line}")

In [ ]:
# Create a comprehensive tube stations dataset
# We'll use the railway stations and identify which ones are tube stations
# by checking proximity to subway lines

# First, let's create a buffered version of subway lines
from shapely.ops import unary_union

# Union all subway lines and create a buffer (100m)
subway_union = unary_union(subway_lines.geometry)
subway_buffer = subway_union.buffer(0.001)  # roughly 100m in degrees

# Find railway stations near subway lines
railway_stations["near_subway"] = railway_stations.geometry.within(subway_buffer)

tube_stations = railway_stations[railway_stations["near_subway"]].copy()

print(f"Total railway stations: {len(railway_stations)}")
print(f"Tube stations (near subway lines): {len(tube_stations)}")
print("\nSample tube stations:")
print(tube_stations[["name", "lon", "lat"]].head(20))

In [ ]:
# Create a clean dataset for tube stations
tube_stations_clean = tube_stations[["name", "osm_id", "lon", "lat"]].copy()
tube_stations_clean = tube_stations_clean.sort_values("name").reset_index(drop=True)

# Create a dictionary for easy lookup
tube_stations_dict = {
    row["name"]: {"osm_id": row["osm_id"], "lat": row["lat"], "lon": row["lon"]}
    for _, row in tube_stations_clean.iterrows()
    if row["name"]  # Filter out any None names
}

print(f"Loaded {len(tube_stations_dict)} tube stations")
print("\nExample stations:")
for i, (name, info) in enumerate(list(tube_stations_dict.items())[:5]):
    print(f"  {name}: ({info['lat']:.6f}, {info['lon']:.6f})")

In [ ]:
graph = london.roads.RoadGraphLoader().load_from_shapefile(
    "/home/cemlyn/Downloads/greater-london-251126-free.shp/gis_osm_roads_free_1.shp"
)

In [ ]:
# list(graph.nodes.keys())[0]
graph.nodes["-0.193124,51.601725"]

In [ ]:
import numpy as np


def haversine(lon1, lat1, lon2, lat2):
    """Calculate the great circle distance between two points on earth (in km)

    All inputs can be scalars or numpy arrays. Arrays will be broadcast together.
    """
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km


In [ ]:
import tqdm

node_ids = list(graph.nodes.keys())
node_lons = np.array([graph.nodes[nid]["lon"] for nid in node_ids])
node_lats = np.array([graph.nodes[nid]["lat"] for nid in node_ids])

station_lons = railway_stations["lon"].values
station_lats = railway_stations["lat"].values

# Process in chunks to avoid memory issues
chunk_size = 8192
closest_station_indices = np.empty(len(node_ids), dtype=np.int64)
closest_station_distances = np.empty(len(node_ids), dtype=np.float64)

with tqdm.tqdm(total=len(node_ids)) as pbar:
    for i in range(0, len(node_ids), chunk_size):
        chunk_end = min(i + chunk_size, len(node_ids))
        chunk_lons = node_lons[i:chunk_end]
        chunk_lats = node_lats[i:chunk_end]

        # Compute distances for this chunk
        distances = haversine(
            chunk_lons[:, np.newaxis],
            chunk_lats[:, np.newaxis],
            station_lons[np.newaxis, :],
            station_lats[np.newaxis, :],
        )
        if distances.shape != (chunk_end - i, station_lons.shape[0]):
            raise ValueError("Fail")

        # Find closest station index for each node in chunk
        indices = distances.argmin(axis=1)
        closest_distances = distances[indices]
        closest_station_indices[i:chunk_end] = distances.argmin(axis=1)
        pbar.update(chunk_end - i)

print(f"Processed {len(node_ids)} nodes in chunks of {chunk_size}")
print(f"Example: Node 0 closest to station index {closest_station_indices[0]}")

In [ ]:
for railway_index in range(railway_stations.shape[0]):
    indices = np.argwhere(closest_station_indices == railway_index).ravel()
    if indices.size > 0:
        closest_sub_index = np.argmin(closest_station_distances[indices])
        index = indices[closest_sub_index]
    else:
        distances = haversine(
            node_lons,
            node_lats,
            station_lons[railway_index],
            station_lats[railway_index],
        )
        index = distances.argmin()
    node_id = node_ids[index]
    graph.nodes[node_id].setdefault("stations", []).append(
        railway_stations.iloc[railway_index].to_dict()
    )

In [ ]:
from collections.abc import Awaitable


async def get_journey_duration(
    tf_client: tfl.api.Tfl,
    source: tuple[float, float],
    target: tuple[float, float],
):
    """Get minimum journey duration for a node. (lat, long)"""
    try:
        results = await tf_client.get_journey_results(
            from_location=source,
            to_location=target,
            arrival_datetime=datetime.datetime.now(tz=datetime.timezone.utc)
            + datetime.timedelta(hours=12),
            modes=[
                tfl.models.ModeId.TUBE,
                tfl.models.ModeId.OVERGROUND,
                tfl.models.ModeId.ELIZABETH_LINE,
                tfl.models.ModeId.DLR,
                tfl.models.ModeId.WALKING,
            ],
            use_multi_modal_call=False,
        )
        if isinstance(results, tfl.models.DisambiguationResult):
            if len(results.to_location_disambiguation.disambiguation_options or []) > 0:
                raise ValueError("Ambigious to location")
            if results.from_location_disambiguation.disambiguation_options is not None:
                disambiguation_options = (
                    results.from_location_disambiguation.disambiguation_options
                )
                best_option = max(disambiguation_options, key=lambda x: x.match_quality)
                return await get_journey_duration(
                    tf_client, (best_option.place.lat, best_option.place.lon), target
                )
            raise ValueError("Ambigious result")
        min_duration_minutes = min(
            sum(leg.duration for leg in journey.legs) for journey in results.journeys
        )
        return min_duration_minutes
    except tfl.exceptions.TflApiError:
        logging.exception(f"API error for node {node_id}")
        return None


async def identify[T, R](key: T, awaitable: Awaitable[R]) -> tuple[T, R]:
    return key, await awaitable

In [ ]:
for node_id, node_attribute in graph.nodes.items():
    node_lat = node_attribute["lat"]
    node_lon = node_attribute["lon"]
    break
node_attribute

In [ ]:
import statistics

target = (51.518088819704815, -0.10794336452296352)
target_lat, target_lon = target

closest_node = None
closest_distance = float("inf")
for node_id, node_attribute in graph.nodes.items():
    node_lat = node_attribute["lat"]
    node_lon = node_attribute["lon"]
    if round(node_lat, 3) == 51.518 and round(node_lon, 3) == -0.108:
        print(node_id)
    distance = haversine(node_lon, node_lat, target_lon, target_lat)
    if distance < closest_distance:
        closest_distance = distance
        closest_node = node_id
if closest_node is None:
    raise ValueError("No close node")

frontier: set[str] = {
    node_id
    for node_id, node_attributes in graph.nodes.items()
    if "stations" in node_attributes
}
frontier.add(closest_node)

max_duration = 30

node_id_durations: dict[str, int | float] = {}
frontier_durations = []

In [ ]:
import pickle
from pathlib import Path

if Path("trainline_road_node_durations.pkl").exists():
    node_id_durations = pickle.loads(
        Path("trainline_road_node_durations.pkl").read_bytes()
    )
else:
    with tqdm.tqdm() as progress_bar:
        progress_bar.update(0)
        while frontier:
            progress_bar.set_description_str(
                " |  ".join(
                    (
                        f"{key}: {value}"
                        for key, value in (
                            ("Frontier Size", len(frontier)),
                            (
                                "Min Duration",
                                min(frontier_durations)
                                if frontier_durations
                                else "N/A",
                            ),
                            (
                                "Mean Duration",
                                round(statistics.mean(frontier_durations))
                                if frontier_durations
                                else "N/A",
                            ),
                            (
                                "Max Duration",
                                max(frontier_durations)
                                if frontier_durations
                                else "N/A",
                            ),
                        )
                    )
                )
                + ". Iteration",
                refresh=True,
            )
            awaitables = [
                identify(
                    node_id,
                    get_journey_duration(
                        tf_client,
                        (graph.nodes[node_id]["lat"], graph.nodes[node_id]["lon"]),
                        target,
                    ),
                )
                for node_id in frontier
            ]
            next_frontier = set()
            frontier_durations.clear()
            async for future in asyncio.as_completed(awaitables):
                node_id, duration = await future
                if duration is not None:
                    frontier_durations.append(duration)
                    node_id_durations[node_id] = min(
                        duration, node_id_durations.get(node_id, float("inf"))
                    )
                    if duration <= max_duration:
                        next_frontier.update(
                            adjacent_node_id
                            for adjacent_node_id in graph.adj[node_id]
                            if adjacent_node_id not in node_id_durations
                        )
                else:
                    node_id_durations[node_id] = float("inf")
                progress_bar.set_description_str(
                    " |  ".join(
                        (
                            f"{key}: {value}"
                            for key, value in (
                                ("Frontier Size", len(frontier)),
                                (
                                    "Min Duration",
                                    min(frontier_durations)
                                    if frontier_durations
                                    else "N/A",
                                ),
                                (
                                    "Mean Duration",
                                    round(statistics.mean(frontier_durations))
                                    if frontier_durations
                                    else "N/A",
                                ),
                                (
                                    "Max Duration",
                                    max(frontier_durations)
                                    if frontier_durations
                                    else "N/A",
                                ),
                            )
                        )
                    )
                    + ". Iteration",
                    refresh=True,
                )
            frontier = next_frontier
            progress_bar.update(1)

In [ ]:
# len(node_id_durations)
# Processed 91817 nodes in 3hrs 14mins

# Path("trainline_road_node_durations.pkl").write_bytes(pickle.dumps(node_id_durations))

In [ ]:
import networkx as nx

subset = {
    node_id
    for node_id, duration in node_id_durations.items()
    if duration <= max_duration
}
subgraph = graph.subgraph(subset)

# Use NetworkX's built-in connected components
sub_graphs = list(nx.weakly_connected_components(subgraph))

print(f"Found {len(sub_graphs)} connected components")
print(f"Largest component has {max(len(sg) for sg in sub_graphs)} nodes")
print(f"Smallest component has {min(len(sg) for sg in sub_graphs)} nodes")

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import scipy.spatial

for _subgraph in tqdm.tqdm(sub_graphs):
    points = np.array(
        [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in _subgraph]
    )
    if len(_subgraph) < 3:
        continue
    hull = scipy.spatial.ConvexHull(points)
    plt.plot(points[hull.vertices, 0], points[hull.vertices, 1], "r--", lw=2)
    plt.plot(points[hull.vertices[0], 0], points[hull.vertices[0], 1], "ro")
    # Connect the first and last point to close the hull
    plt.plot(
        [points[hull.vertices[-1], 0], points[hull.vertices[0], 0]],
        [points[hull.vertices[-1], 1], points[hull.vertices[0], 1]],
        "r--",
        lw=2,
    )
all_points = np.array(
    [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in subset]
)
plt.scatter(all_points[:, 0], all_points[:, 1], s=0.1)

In [ ]:
import scipy.spatial

_subgraph = max(sub_graphs, key=len)
points = np.array(
    [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in _subgraph]
)
hull = scipy.spatial.ConvexHull(points)
plt.plot(points[hull.vertices, 0], points[hull.vertices, 1], "r--", lw=2)
plt.plot(points[hull.vertices[0], 0], points[hull.vertices[0], 1], "ro")
# Connect the first and last point to close the hull
plt.plot(
    [points[hull.vertices[-1], 0], points[hull.vertices[0], 0]],
    [points[hull.vertices[-1], 1], points[hull.vertices[0], 1]],
    "r--",
    lw=2,
)
all_points = np.array(
    [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in subset]
)
plt.scatter(all_points[:, 0], all_points[:, 1], s=0.1)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.stats import gaussian_kde

if False:
    plt.figure()
    for subgraph in sub_graphs:
        # Get points for this cluster
        cluster_points = np.array(
            [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in subgraph]
        )

        if len(cluster_points) < 10:  # Skip very small clusters or handle as points
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], color="blue")
            continue

        # Plot blue points
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], color="blue", s=0.01)

        # Compute KDE with adjustable bandwidth (smaller = tighter, more detailed; try 0.01-0.1 for your scale)
        kde = gaussian_kde(cluster_points.T, bw_method=0.05)

        # Create evaluation grid (higher resolution for smoother contours)
        xmin, xmax = (
            cluster_points[:, 0].min() - 0.005,
            cluster_points[:, 0].max() + 0.005,
        )
        ymin, ymax = (
            cluster_points[:, 1].min() - 0.005,
            cluster_points[:, 1].max() + 0.005,
        )
        xi, yi = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]  # 200x200 grid for detail
        coords = np.vstack([xi.ravel(), yi.ravel()])
        z = kde.evaluate(coords).reshape(xi.shape)

        # Determine contour level to enclose points (tune level_factor: 0.01-0.1 for outer, higher for tighter)
        densities_at_points = kde.evaluate(cluster_points.T)
        min_density = np.min(densities_at_points)
        level = (
            min_density * 0.5
        )  # Start with 0.5; decrease to loosen, increase to tighten

        # Plot red dashed contour
        plt.contour(xi, yi, z, levels=[level], colors="red", linestyles="dashed")

    # Plot red outliers (assuming you have a list of outlier points)
    # outliers = np.array([...])  # Your red points
    # plt.scatter(outliers[:, 0], outliers[:, 1], color='red')

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

In [ ]:
plt.get_fignums()

In [ ]:
import matplotlib.pyplot as plt

# largest_subgraph = max(sub_graphs, key=len)

# nx.draw(graph.subgraph(largest_subgraph))

plt.hist(
    [len(sg) for sg in sub_graphs],
    # bins=range(1, max(len(sg) for sg in sub_graphs) + 1),
)

In [ ]:
# nx.connected_components(graph, [graph.nodes["-0.043770,51.538328"], graph.nodes["-0.025515,51.527226"]])
subset = {
    node_id
    for node_id, duration in node_id_durations.items()
    if duration <= max_duration
}
subgraph = graph.subgraph(subset)


In [ ]:
for node_id, node_attribute in graph.nodes.items():
    duration = node_id_durations.get(node_id, float("inf"))
    node_attribute["duration"] = duration

isochrone_subgraph = nx.ego_graph(
    graph, closest_node, radius=max_duration, undirected=True, distance="duration"
)

In [ ]:
isochrone_subgraph.number_of_edges()

In [ ]:
sub_graphs = list(nx.weakly_connected_components(isochrone_subgraph))
len(sub_graphs)

In [ ]:
import scipy.spatial

for _subgraph in tqdm.tqdm(sub_graphs):
    points = np.array(
        [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in _subgraph]
    )
    if len(_subgraph) < 3:
        continue
    hull = scipy.spatial.ConvexHull(points)
    plt.plot(points[hull.vertices, 0], points[hull.vertices, 1], "r--", lw=2)
    plt.plot(points[hull.vertices[0], 0], points[hull.vertices[0], 1], "ro")
    # Connect the first and last point to close the hull
    plt.plot(
        [points[hull.vertices[-1], 0], points[hull.vertices[0], 0]],
        [points[hull.vertices[-1], 1], points[hull.vertices[0], 1]],
        "r--",
        lw=2,
    )
all_points = np.array(
    [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in subset]
)
plt.scatter(all_points[:, 0], all_points[:, 1], s=0.1)

In [ ]:
# pos = nx.spring_layout(isochrone_subgraph)
nx.draw(isochrone_subgraph)

In [ ]:
def get_boundary_adjacent_nodes(
    parent_graph: nx.DiGraph,
    subgraph_nodes: set[str],
) -> tuple[set[str], set[str]]:
    """
    Get the boundary nodes of the subgraph and their immediate neighbors outside the subgraph.

    Returns:
        boundary_nodes: Nodes in subgraph that have neighbors outside subgraph
        adjacent_nodes: Nodes outside subgraph that are direct neighbors of boundary nodes
    """
    boundary_nodes = set()
    adjacent_nodes = set()

    for node in subgraph_nodes:
        neighbors = set(parent_graph.successors(node)) | set(
            parent_graph.predecessors(node)
        )
        outside_neighbors = neighbors - subgraph_nodes
        if outside_neighbors:
            boundary_nodes.add(node)
            adjacent_nodes.update(outside_neighbors)

    return boundary_nodes, adjacent_nodes


def get_hull_node_ids(
    graph: nx.DiGraph,
    subgraph_nodes: set[str],
) -> list[str]:
    """Get the node IDs corresponding to convex hull vertices in order."""
    points = np.array(
        [(graph.nodes[nid]["lon"], graph.nodes[nid]["lat"]) for nid in subgraph_nodes]
    )
    node_list = list(subgraph_nodes)

    if len(points) < 3:
        return node_list

    hull = scipy.spatial.ConvexHull(points)
    return [node_list[i] for i in hull.vertices]


In [ ]:
# import concurrent.futures

# largest_subgraph_nodes = max(sub_graphs, key=len)
# hull_nodes = get_hull_node_ids(graph, largest_subgraph_nodes)

# _subgraph = graph.subgraph(largest_subgraph_nodes)

# start = hull_nodes[0]
# end = hull_nodes[1]


# def haversine_node(graph: nx.DiGraph, start: str, end: str) -> float:
#     try:
#         length = nx.shortest_path_length(_subgraph, start, end)
#     except nx.NetworkXNoPath:
#         try:
#             length = nx.shortest_path_length(graph, start, end)
#         except nx.NetworkXNoPath:
#             return float("inf")
#     return length


# best_length = float("inf")
# best = None
# with tqdm.tqdm(largest_subgraph_nodes) as pbar:
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         for node_id, length in executor.map(
#             lambda nid: (nid, haversine_node(graph, start, nid)),
#             largest_subgraph_nodes,
#         ):
#             if length < best_length:
#                 best_length = length
#                 best = node_id
#             pbar.update(1)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
from shapely.geometry import LineString, Point, Polygon

# def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
multi_graph = nx.MultiDiGraph(graph)
# WGS84 coordinate reference system which is written as EPSG:4326
multi_graph.graph["crs"] = "EPSG:4326"
for node_id in multi_graph.nodes:
    multi_graph.nodes[node_id]["x"] = multi_graph.nodes[node_id]["lon"]
    multi_graph.nodes[node_id]["y"] = multi_graph.nodes[node_id]["lat"]


G = multi_graph
edge_buff = 25
node_buff = 0.1
infill = True


isochrone_polys = []

new_graph = nx.Graph()
new_graph.add_node(
    closest_node, x=G.nodes[closest_node]["lon"], y=G.nodes[closest_node]["lat"]
)
for node_id in G.nodes:
    duration = node_id_durations.get(node_id, float("inf"))
    new_graph.add_node(node_id, x=G.nodes[node_id]["lon"], y=G.nodes[node_id]["lat"])
    new_graph.add_edge(closest_node, node_id, duration=duration)

nG = nx.MultiGraph(new_graph)
nG.graph["crs"] = "EPSG:4326"

tmp_subgraph = nx.ego_graph(
    nG, closest_node, radius=max_duration, undirected=False, distance="duration"
)

subgraph = nx.subgraph(
    G,
    [
        node
        for node, data in tmp_subgraph.nodes(data=True)
        # if data.get("duration", float("inf")) <= max_duration
    ],
)

node_points = [
    Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)
]
nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
nodes_gdf = nodes_gdf.set_index("id")

edge_lines = []
for n_fr, n_to in subgraph.edges():
    f = nodes_gdf.loc[n_fr].geometry
    t = nodes_gdf.loc[n_to].geometry
    edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
    edge_lines.append(edge_lookup)

# TODO?
# n = nodes_gdf.buffer(node_buff).geometry
# e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
# all_gs = list(n) + list(e)
# new_iso = gpd.GeoSeries(all_gs).union_all()
new_iso = gpd.GeoSeries(
    list(nodes_gdf.geometry) + list(gpd.GeoSeries(edge_lines).geometry)
).union_all()

# try to fill in surrounded areas so shapes will appear solid and
# blocks without white space inside them
if infill:
    new_iso = Polygon(new_iso.exterior)
    isochrone_polys.append(new_iso)
else:
    isochrone_polys.append(new_iso)


# make the isochrone polygons
# TODO: Why does node_buff == 0 cause empty polygons on `n = nodes_gdf.buffer(node_buff).geometry`


In [ ]:
new_iso.buffer(50)

In [ ]:
gdf = gpd.GeoDataFrame(geometry=[new_iso])
import osmnx as ox

iso_colors = ox.plot.get_colors(n=1, cmap="plasma", start=0)
# # plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot.plot_graph(
    multi_graph,
    show=False,
    close=False,
    edge_color="#999999",
    edge_alpha=0.2,
    node_size=0,
    figsize=(16, 16),
)
gdf.plot(ax=ax, color=iso_colors, ec="none", alpha=0.6, zorder=-1)

In [ ]:
def make_iso_polys(G, subgraph):
    node_points = [
        Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)
    ]
    nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index("id")

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lookup = G.get_edge_data(n_fr, n_to).get("geometry", LineString([f, t]))
        edge_lines.append(edge_lookup)

    return gpd.GeoSeries(
        list(nodes_gdf.buffer(0).geometry)
        + list(gpd.GeoSeries(edge_lines).buffer(25).geometry)
    ).union_all()

In [ ]:
# gdf.iloc[0].geometry.is_closed
# gdf.iloc[0].geometry.segmentize()

for node_id in graph.nodes:
    graph.nodes[node_id]["x"] = graph.nodes[node_id]["lon"]
    graph.nodes[node_id]["y"] = graph.nodes[node_id]["lat"]

separated_subgraphs = list(nx.weakly_connected_components(subgraph))

# nx.draw(graph.subgraph(separated_subgraphs[2]))
iso = make_iso_polys(graph, graph.subgraph(separated_subgraphs[0]))
iso.exterior

In [ ]:
roads_gdf = gpd.read_file(
    "/home/cemlyn/Downloads/greater-london-251126-free.shp/gis_osm_roads_free_1.shp"
)

In [ ]:
roads_gdf.union_all()

In [ ]:
iso

In [ ]:
_sub_graph = graph.subgraph(separated_subgraphs[2])

In [ ]:
# roads_gdf.geometry.contains()

for node_attribute in _sub_graph.nodes.values():
    point = Point((node_attribute["lon"], node_attribute["lat"]))
    containing_roads = roads_gdf[roads_gdf.geometry.contains(point)]
    if not containing_roads.empty:
        print(containing_roads)